## Model 3.2

In [1]:
# pip install hyperopt

In [2]:
from ortools.sat.python import cp_model
import random
from hyperopt import fmin, tpe, hp, Trials
import itertools

In [15]:
def generate_home_away_patterns(num_teams, num_rounds):
    """Generate fair home-away patterns that account for byes if num_teams is odd."""
    patterns = []
    num_byes = num_rounds // num_teams  # Approximate number of byes per team if odd teams
    
    for i in range(num_teams):
        pattern = []
        bye_rounds = set(random.sample(range(num_rounds), num_byes)) if num_teams % 2 == 1 else set()
        
        for r in range(num_rounds):
            if r in bye_rounds:
                pattern.append('B')  # 'B' for Bye
            elif (i + r) % 2 == 0:
                pattern.append('H')  # Home game
            else:
                pattern.append('A')  # Away game
        patterns.append(pattern)
    return patterns

In [16]:
def generate_teams(num_teams):
    """
    Generates a dictionary of teams with randomly assigned inter-league rankings.
    
    Each team is represented by an integer ID (0 to num_teams-1) and a unique ranking
    (randomly shuffled from 1 to num_teams, where a lower number might indicate a higher rank).
    """
    rankings = list(range(1, num_teams + 1))
    random.shuffle(rankings)
    teams = {i: rankings[i] for i in range(num_teams)}
    return teams

num_teams = 7
teams = generate_teams(num_teams)
print("Team Rankings:", teams)


Team Rankings: {0: 2, 1: 3, 2: 6, 3: 1, 4: 5, 5: 7, 6: 4}


In [5]:
# from ortools.sat.python import cp_model
# import random
# import itertools

# def build_model(num_teams, w_consec, w_start, w_end, w_carry, w_opp, w_high):
#     """Builds the CP model for the Belgian Pro League schedule with parameterized penalty weights."""
#     model = cp_model.CpModel()

#     # Determine number of rounds dynamically.
#     if num_teams % 2 == 0:
#         num_rounds = 2 * (num_teams - 1)
#     else:
#         num_rounds = 2 * num_teams

#     # Decision variables.
#     x = {}  # x[i, j, r] = 1 if team i plays team j in round r.
#     b = {}  # b[i, r] = 1 if team i has a bye in round r.
#     for i in range(num_teams):
#         for r in range(num_rounds):
#             b[i, r] = model.NewBoolVar(f'b_{i}_{r}')
#             for j in range(num_teams):
#                 if i != j:
#                     x[i, j, r] = model.NewBoolVar(f'x_{i}_{j}_{r}')

#     # Hard Constraint 1: Each team plays exactly one match (or has a bye) per round.
#     for r in range(num_rounds):
#         for i in range(num_teams):
#             model.Add(sum(x[i, j, r] + x[j, i, r] for j in range(num_teams) if i != j) + b[i, r] == 1)

#     # Hard Constraint 2: Exactly one bye per round (if odd number of teams).
#     if num_teams % 2 == 1:
#         for r in range(num_rounds):
#             model.Add(sum(b[i, r] for i in range(num_teams)) == 1)

#     # Hard Constraint 3: Each pair of teams plays exactly twice.
#     for i in range(num_teams):
#         for j in range(num_teams):
#             if i != j:
#                 model.Add(sum(x[i, j, r] + x[j, i, r] for r in range(num_rounds)) == 2)

#     # Hard Constraint 4: Equal home/away balance.
#     for i in range(num_teams):
#         total_home = sum(x[i, j, r] for j in range(num_teams) if i != j for r in range(num_rounds))
#         total_away = sum(x[j, i, r] for j in range(num_teams) if i != j for r in range(num_rounds))
#         model.Add(total_home == total_away)

#     M = 3  # Big-M constant.

#     # Soft Constraint 1: Minimizing excessive consecutive matches.
#     v_consec = {}
#     for i in range(num_teams):
#         for r in range(num_rounds - 2):
#             v_consec[i, r] = model.NewBoolVar(f'v_consec_{i}_{r}')
#             home_sum = (sum(x[i, j, r] for j in range(num_teams) if i != j) +
#                         sum(x[i, j, r+1] for j in range(num_teams) if i != j) +
#                         sum(x[i, j, r+2] for j in range(num_teams) if i != j))
#             away_sum = (sum(x[j, i, r] for j in range(num_teams) if i != j) +
#                         sum(x[j, i, r+1] for j in range(num_teams) if i != j) +
#                         sum(x[j, i, r+2] for j in range(num_teams) if i != j))
#             model.Add(home_sum <= 2 + M * v_consec[i, r])
#             model.Add(home_sum >= 3 * v_consec[i, r])
#             model.Add(away_sum <= 2 + M * v_consec[i, r])
#             model.Add(away_sum >= 3 * v_consec[i, r])

#     # Soft Constraint 2: Start/End Fairness.
#     v_start = {}
#     v_end = {}
#     for i in range(num_teams):
#         v_start[i] = model.NewBoolVar(f'v_start_{i}')
#         v_end[i] = model.NewBoolVar(f'v_end_{i}')
#         home_first = (sum(x[i, j, 0] for j in range(num_teams) if i != j) +
#                       sum(x[i, j, 1] for j in range(num_teams) if i != j))
#         away_first = (sum(x[j, i, 0] for j in range(num_teams) if i != j) +
#                       sum(x[j, i, 1] for j in range(num_teams) if i != j))
#         model.Add(home_first <= 1 + M * v_start[i])
#         model.Add(away_first <= 1 + M * v_start[i])
#         model.Add(home_first + away_first >= 2 * v_start[i])
#         home_last = (sum(x[i, j, num_rounds-2] for j in range(num_teams) if i != j) +
#                      sum(x[i, j, num_rounds-1] for j in range(num_teams) if i != j))
#         away_last = (sum(x[j, i, num_rounds-2] for j in range(num_teams) if i != j) +
#                      sum(x[j, i, num_rounds-1] for j in range(num_teams) if i != j))
#         model.Add(home_last <= 1 + M * v_end[i])
#         model.Add(away_last <= 1 + M * v_end[i])
#         model.Add(home_last + away_last >= 2 * v_end[i])

#     # Soft Constraint 3: Carry-Over Effect Balance (Conceptual Placeholder).
#     v_carry = {}
#     for i in range(num_teams):
#         for r in range(num_rounds - 1):
#             v_carry[i, r] = model.NewIntVar(0, 10, f'v_carry_{i}_{r}')
#             # Placeholder: implement actual constraints using opponent rankings.

#     # Soft Constraint 4: Opponent Quality Distribution (Conceptual Placeholder).
#     v_opp = {}
#     for i in range(num_teams):
#         v_opp[i] = model.NewIntVar(0, 100, f'v_opp_{i}')
#         # Placeholder: implement the calculation of deviation from an ideal total ranking.

#     # Soft Constraint 5: Even Distribution of High-Profile Matches (Optional, Conceptual Placeholder).
#     v_high = {}
#     for r in range(num_rounds):
#         v_high[r] = model.NewIntVar(0, num_teams, f'v_high_{r}')
#         # Placeholder: implement based on high-profile match criteria.

#     # Objective Function: minimize the weighted sum of soft constraint violations.
#     objective_terms = []
#     for i in range(num_teams):
#         for r in range(num_rounds - 2):
#             objective_terms.append(w_consec * v_consec[i, r])
#     for i in range(num_teams):
#         objective_terms.append(w_start * v_start[i])
#         objective_terms.append(w_end * v_end[i])
#     for i in range(num_teams):
#         for r in range(num_rounds - 1):
#             objective_terms.append(w_carry * v_carry[i, r])
#     for i in range(num_teams):
#         objective_terms.append(w_opp * v_opp[i])
#     for r in range(num_rounds):
#         objective_terms.append(w_high * v_high[r])
#     model.Minimize(sum(objective_terms))

#     return model, num_rounds, x, b, v_consec, v_start, v_end, v_carry, v_opp, v_high

# def belgian_pro_league_schedule(num_teams, weights):
#     """
#     Builds and solves the model using the given penalty weights.
#     `weights` is a tuple: (w_consec, w_start, w_end, w_carry, w_opp, w_high).
#     Returns the schedule and the solver.
#     """
#     model, num_rounds, x, b, v_consec, v_start, v_end, v_carry, v_opp, v_high = build_model(num_teams, *weights)
#     solver = cp_model.CpSolver()
#     solver.parameters.max_time_in_seconds = 120
#     solver.parameters.log_search_progress = True
#     status = solver.Solve(model)
#     schedule = []
#     if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
#         for r in range(num_rounds):
#             round_matches = []
#             bye_teams = set(range(num_teams))
#             for i in range(num_teams):
#                 for j in range(num_teams):
#                     if i != j and solver.Value(x[i, j, r]) == 1:
#                         round_matches.append((i, j))
#                         bye_teams.discard(i)
#                         bye_teams.discard(j)
#             if bye_teams:
#                 round_matches.append(("Bye", list(bye_teams)[0]))
#             schedule.append(round_matches)
#     else:
#         print("No feasible solution found.")
#     return schedule, solver

# def evaluate_weights(num_teams, weights):
#     """
#     Builds and solves the model using the given weights, then returns the objective value.
#     """
#     model, num_rounds, x, b, v_consec, v_start, v_end, v_carry, v_opp, v_high = build_model(num_teams, *weights)
#     solver = cp_model.CpSolver()
#     solver.parameters.max_time_in_seconds = 120
#     status = solver.Solve(model)
#     if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
#         return solver.ObjectiveValue()
#     else:
#         return float('inf')


In [6]:
# import itertools
# from ortools.sat.python import cp_model

# def build_model(num_teams, w_consec, w_start, w_end, w_carry, w_opp, w_high):
#     model = cp_model.CpModel()

#     # Determine number of rounds dynamically.
#     if num_teams % 2 == 0:
#         num_rounds = 2 * (num_teams - 1)
#     else:
#         num_rounds = 2 * num_teams

#     # Decision variables.
#     x = {}  # x[i, j, r] = 1 if team i plays team j in round r.
#     b = {}  # b[i, r] = 1 if team i has a bye in round r.
#     for i in range(num_teams):
#         for r in range(num_rounds):
#             b[i, r] = model.NewBoolVar(f'b_{i}_{r}')
#             for j in range(num_teams):
#                 if i != j:
#                     x[i, j, r] = model.NewBoolVar(f'x_{i}_{j}_{r}')

#     # Hard Constraint 1: Each team plays once per round or has a bye.
#     for r in range(num_rounds):
#         for i in range(num_teams):
#             model.Add(sum(x[i, j, r] + x[j, i, r] for j in range(num_teams) if i != j) + b[i, r] == 1)

#     # Hard Constraint 2: Exactly one bye per round (only for odd number of teams).
#     if num_teams % 2 == 1:
#         for r in range(num_rounds):
#             model.Add(sum(b[i, r] for i in range(num_teams)) == 1)

#     # Hard Constraint 3: Each team plays every other team exactly twice.
#     for i in range(num_teams):
#         for j in range(num_teams):
#             if i != j:
#                 model.Add(sum(x[i, j, r] + x[j, i, r] for r in range(num_rounds)) == 2)

#     # Hard Constraint 4: Equal home/away balance.
#     for i in range(num_teams):
#         total_home = sum(x[i, j, r] for j in range(num_teams) if i != j for r in range(num_rounds))
#         total_away = sum(x[j, i, r] for j in range(num_teams) if i != j for r in range(num_rounds))
#         model.Add(total_home == total_away)

#     # ---------------- Soft Constraints ----------------

#     M = 3  # Big-M constant.

#     # Soft Constraint 1: Minimizing excessive consecutive matches.
#     v_consec = {}
#     for i in range(num_teams):
#         for r in range(num_rounds - 2):
#             v_consec[i, r] = model.NewBoolVar(f'v_consec_{i}_{r}')
#             home_sum = (sum(x[i, j, r] for j in range(num_teams) if i != j) +
#                         sum(x[i, j, r+1] for j in range(num_teams) if i != j) +
#                         sum(x[i, j, r+2] for j in range(num_teams) if i != j))
#             away_sum = (sum(x[j, i, r] for j in range(num_teams) if i != j) +
#                         sum(x[j, i, r+1] for j in range(num_teams) if i != j) +
#                         sum(x[j, i, r+2] for j in range(num_teams) if i != j))
#             # Force violation variable to be 1 if home_sum (or away_sum) equals 3.
#             model.Add(home_sum <= 2 + M * v_consec[i, r])
#             model.Add(home_sum >= 3 * v_consec[i, r])
#             model.Add(away_sum <= 2 + M * v_consec[i, r])
#             model.Add(away_sum >= 3 * v_consec[i, r])

#     # Soft Constraint 2: Start/End Fairness.
#     v_start = {}
#     v_end = {}
#     for i in range(num_teams):
#         v_start[i] = model.NewBoolVar(f'v_start_{i}')
#         v_end[i] = model.NewBoolVar(f'v_end_{i}')
#         home_first = (sum(x[i, j, 0] for j in range(num_teams) if i != j) +
#                       sum(x[i, j, 1] for j in range(num_teams) if i != j))
#         away_first = (sum(x[j, i, 0] for j in range(num_teams) if i != j) +
#                       sum(x[j, i, 1] for j in range(num_teams) if i != j))
#         model.Add(home_first <= 1 + M * v_start[i])
#         model.Add(away_first <= 1 + M * v_start[i])
#         model.Add(home_first + away_first >= 2 * v_start[i])
#         home_last = (sum(x[i, j, num_rounds-2] for j in range(num_teams) if i != j) +
#                      sum(x[i, j, num_rounds-1] for j in range(num_teams) if i != j))
#         away_last = (sum(x[j, i, num_rounds-2] for j in range(num_teams) if i != j) +
#                      sum(x[j, i, num_rounds-1] for j in range(num_teams) if i != j))
#         model.Add(home_last <= 1 + M * v_end[i])
#         model.Add(away_last <= 1 + M * v_end[i])
#         model.Add(home_last + away_last >= 2 * v_end[i])

#     # Soft Constraint 3: Carry-Over Effect Balance (Conceptual Placeholder).
#     v_carry = {}
#     T = 5  # Example threshold.
#     for i in range(num_teams):
#         for r in range(num_rounds - 1):
#             v_carry[i, r] = model.NewIntVar(0, 10, f'v_carry_{i}_{r}')
#             # (Placeholder: Implement actual constraint linking opponent rankings.)

#     # Soft Constraint 4: Opponent Quality Distribution (Conceptual Placeholder).
#     v_opp = {}
#     for i in range(num_teams):
#         v_opp[i] = model.NewIntVar(0, 100, f'v_opp_{i}')
#         # (Placeholder: Implement calculation of total opponent ranking deviation.)

#     # Soft Constraint 5: Even Distribution of High-Profile Matches (Optional, Conceptual).
#     v_high = {}
#     for r in range(num_rounds):
#         v_high[r] = model.NewIntVar(0, num_teams, f'v_high_{r}')
#         # (Placeholder: Implement high-profile match criteria.)

#     # ---------------- Objective Function ----------------

#     objective_terms = []
#     for i in range(num_teams):
#         for r in range(num_rounds - 2):
#             objective_terms.append(w_consec * v_consec[i, r])
#     for i in range(num_teams):
#         objective_terms.append(w_start * v_start[i])
#         objective_terms.append(w_end * v_end[i])
#     for i in range(num_teams):
#         for r in range(num_rounds - 1):
#             objective_terms.append(w_carry * v_carry[i, r])
#     for i in range(num_teams):
#         objective_terms.append(w_opp * v_opp[i])
#     for r in range(num_rounds):
#         objective_terms.append(w_high * v_high[r])
    
#     model.Minimize(sum(objective_terms))
#     return model, num_rounds, x, b, v_consec, v_start, v_end, v_carry, v_opp, v_high

# # ---------------- Grid Search for Sensitivity Analysis ----------------

# # Define candidate weight values for each parameter.
# w_consec_values = [1, 3, 5]
# w_start_values  = [1, 3]
# w_end_values    = [1, 3]
# w_carry_values  = [1, 3]
# w_opp_values    = [1, 2]
# w_high_values   = [0, 1]  # Optional

# # Create a grid of parameters using itertools.product.
# param_grid = list(itertools.product(w_consec_values, w_start_values, w_end_values,
#                                     w_carry_values, w_opp_values, w_high_values))

# results = []

# for params in param_grid:
#     w_consec, w_start, w_end, w_carry, w_opp, w_high = params
#     model, num_rounds, x, b, v_consec, v_start, v_end, v_carry, v_opp, v_high = build_model(num_teams, w_consec, w_start, w_end, w_carry, w_opp, w_high)
#     solver = cp_model.CpSolver()
#     status = solver.Solve(model)
    
#     if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
#         obj_value = solver.ObjectiveValue()
#         results.append((params, obj_value))
#     else:
#         results.append((params, None))

# # Print the grid search results.
# print("Grid Search Results:")
# for params, obj_value in results:
#     print(f"Parameters (w_consec, w_start, w_end, w_carry, w_opp, w_high): {params}, Objective Value: {obj_value}")


In [17]:
def belgian_pro_league_schedule(num_teams):
    """Creates a valid Belgian Pro League schedule for any number of teams (even or odd)."""    
    global solver, num_rounds, v_consec, v_start, v_end, v_carry, v_opp, v_high

    model = cp_model.CpModel()

    # **Determine number of rounds dynamically**
    if num_teams % 2 == 0:
        num_rounds = 2 * (num_teams - 1)  # Even number of teams
    else:
        num_rounds = 2 * num_teams  # Odd number of teams (Extra rounds for flexibility)

    # **Decision Variables**
    x = {}  # Match variable: x[i, j, r] = 1 if team i plays against team j in round r
    b = {}  # Bye variable: b[i, r] = 1 if team i has a bye in round r

    for i in range(num_teams):
        for r in range(num_rounds):
            b[i, r] = model.NewBoolVar(f'b_{i}_{r}')
            for j in range(num_teams):
                if i != j:
                    x[i, j, r] = model.NewBoolVar(f'x_{i}_{j}_{r}')

    # **Constraint 1: Each team plays once per round OR has a bye**
    for r in range(num_rounds):
        for i in range(num_teams):
            model.Add(sum(x[i, j, r] + x[j, i, r] for j in range(num_teams) if i != j) + b[i, r] == 1)

    # **Constraint 2: Exactly one bye per round (Only for odd-numbered teams)**
    if num_teams % 2 == 1:
        for r in range(num_rounds):
            model.Add(sum(b[i, r] for i in range(num_teams)) == 1)  # Exactly one bye per round

    # **Constraint 3: Each team plays every other team exactly twice (home & away)**
    for i in range(num_teams):
        for j in range(num_teams):
            if i != j:
                model.Add(sum(x[i, j, r] + x[j, i, r] for r in range(num_rounds)) == 2)

    # **Constraint 4: Ensure Each Team Has an Equal Home/Away Balance**
    for i in range(num_teams):
        total_home_games = sum(x[i, j, r] for j in range(num_teams) if i != j for r in range(num_rounds))
        total_away_games = sum(x[j, i, r] for j in range(num_teams) if i != j for r in range(num_rounds))
        model.Add(total_home_games == total_away_games)



    M = 3  # Big-M constant

    ### Soft Constraint 1: Minimizing Excessive Consecutive Matches
    # For each team i and for each block of 3 consecutive rounds, we want:
    #   - If the team plays 3 consecutive home games (or 3 consecutive away games),
    #     then v_consec[i, r] must be 1.
    #   - Otherwise, v_consec[i, r] must be 0.
    v_consec = {}
    for i in range(num_teams):
        for r in range(num_rounds - 2):
            v_consec[i, r] = model.NewBoolVar(f'v_consec_{i}_{r}')
            
            # Compute the number of home games in rounds r, r+1, r+2 for team i.
            home_sum = (
                sum(x[i, j, r] for j in range(num_teams) if i != j) +
                sum(x[i, j, r+1] for j in range(num_teams) if i != j) +
                sum(x[i, j, r+2] for j in range(num_teams) if i != j)
            )
            # Compute the number of away games in the same rounds.
            away_sum = (
                sum(x[j, i, r] for j in range(num_teams) if i != j) +
                sum(x[j, i, r+1] for j in range(num_teams) if i != j) +
                sum(x[j, i, r+2] for j in range(num_teams) if i != j)
            )
            
            # If v_consec[i, r] = 0, then home_sum and away_sum must be <=2.
            # If v_consec[i, r] = 1, then home_sum (or away_sum) must be at least 3.
            model.Add(home_sum <= 2 + M * v_consec[i, r])
            model.Add(home_sum >= 3 * v_consec[i, r])
            model.Add(away_sum <= 2 + M * v_consec[i, r])
            model.Add(away_sum >= 3 * v_consec[i, r])

    ### Soft Constraint 2: Start/End Fairness
    # For each team i, we want to flag a violation if the first two rounds or the last two rounds
    # are entirely home or entirely away.
    v_start = {}
    v_end = {}
    for i in range(num_teams):
        v_start[i] = model.NewBoolVar(f'v_start_{i}')
        v_end[i] = model.NewBoolVar(f'v_end_{i}')
        
        # First two rounds:
        home_first = (
            sum(x[i, j, 0] for j in range(num_teams) if i != j) +
            sum(x[i, j, 1] for j in range(num_teams) if i != j)
        )
        away_first = (
            sum(x[j, i, 0] for j in range(num_teams) if i != j) +
            sum(x[j, i, 1] for j in range(num_teams) if i != j)
        )
        # If either home_first or away_first equals 2, then violation occurs.
        # Enforce that when v_start[i] = 0, both home_first and away_first must be ≤ 1;
        # and when v_start[i] = 1, their sum must be at least 2.
        model.Add(home_first <= 1 + M * v_start[i])
        model.Add(away_first <= 1 + M * v_start[i])
        model.Add(home_first + away_first >= 2 * v_start[i])
        
        # Last two rounds:
        home_last = (
            sum(x[i, j, num_rounds-2] for j in range(num_teams) if i != j) +
            sum(x[i, j, num_rounds-1] for j in range(num_teams) if i != j)
        )
        away_last = (
            sum(x[j, i, num_rounds-2] for j in range(num_teams) if i != j) +
            sum(x[j, i, num_rounds-1] for j in range(num_teams) if i != j)
        )
        model.Add(home_last <= 1 + M * v_end[i])
        model.Add(away_last <= 1 + M * v_end[i])
        model.Add(home_last + away_last >= 2 * v_end[i])

    ### Soft Constraint 3: Carry-Over Effect Balance (Conceptual Placeholder)
    v_carry = {}
    T = 5  # Example threshold for the sum of opponent rankings
    for i in range(num_teams):
        for r in range(num_rounds - 1):
            v_carry[i, r] = model.NewIntVar(0, 10, f'v_carry_{i}_{r}')
            # You will need to define auxiliary expressions for the opponent rankings
            # and then enforce:
            # model.Add(v_carry[i, r] >= (opponent_ranking_sum - T))
            # (Refine this based on your model structure.)

    ### Soft Constraint 4: Opponent Quality Distribution (Conceptual Placeholder)
    v_opp = {}
    for i in range(num_teams):
        v_opp[i] = model.NewIntVar(0, 100, f'v_opp_{i}')
        # You should compute the total opponent ranking for team i and
        # compare it to an ideal total. Then enforce:
        # model.Add(v_opp[i] >= Abs(total_opponent_ranking(i) - ideal_total))
        # (This requires additional auxiliary variables and constraints.)

    ### Soft Constraint 5: Even Distribution of High-Profile Matches (Optional, Conceptual Placeholder)
    v_high = {}
    for r in range(num_rounds):
        v_high[r] = model.NewIntVar(0, num_teams, f'v_high_{r}')
        # Define the number of high-profile matches in round r based on team rankings,
        # then enforce:
        # model.Add(v_high[r] >= (high_matches(r) - threshold))
        # (Refine this based on your criteria.)





    # Define penalty weights (adjust these values as needed)
    w_consec = 5
    w_start = 3
    w_end = 3
    w_carry = 4
    w_opp = 2
    w_high = 1  # Optional constraint weight

    objective_terms = []

    # Add penalties for excessive consecutive matches
    for i in range(num_teams):
        for r in range(num_rounds - 2):
            objective_terms.append(w_consec * v_consec[i, r])

    # Add penalties for start and end imbalance violations
    for i in range(num_teams):
        objective_terms.append(w_start * v_start[i])
        objective_terms.append(w_end * v_end[i])

    # Add penalties for carry-over effect violations
    for i in range(num_teams):
        for r in range(num_rounds - 1):
            objective_terms.append(w_carry * v_carry[i, r])

    # Add penalties for opponent quality imbalance
    for i in range(num_teams):
        objective_terms.append(w_opp * v_opp[i])

    # Add penalties for high-profile match imbalance (optional)
    for r in range(num_rounds):
        objective_terms.append(w_high * v_high[r])

    # Set the model objective to minimize the total weighted penalty
    model.Minimize(sum(objective_terms))

    # Proceed to solve the model
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 120
    solver.parameters.log_search_progress = True
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        schedule = []
        for r in range(num_rounds):
            round_matches = []
            bye_teams = set(range(num_teams))  # Track which teams are idle
            for i in range(num_teams):
                for j in range(num_teams):
                    if i != j and solver.Value(x[i, j, r]) == 1:
                        round_matches.append((i, j))
                        bye_teams.discard(i)
                        bye_teams.discard(j)
            # Add bye team(s)
            if bye_teams:
                round_matches.append(("Bye", list(bye_teams)[0]))
            schedule.append(round_matches)
        return schedule
    else:
        print("No feasible solution found.")
        return None


In [8]:
# # Define an evaluation function that builds and solves the model for a given set of weights.
# def evaluate_weights(num_teams, weights):
#     """
#     Builds the CP model with the given penalty weights, solves it, and returns the objective value.
#     The weights parameter is a tuple: (w_consec, w_start, w_end, w_carry, w_opp, w_high).
#     """
#     # Build the model. Assume build_model returns:
#     # model, num_rounds, x, b, v_consec, v_start, v_end, v_carry, v_opp, v_high.
#     model, num_rounds, x, b, v_consec, v_start, v_end, v_carry, v_opp, v_high = build_model(num_teams, *weights)
    
#     solver = cp_model.CpSolver()
#     status = solver.Solve(model)
#     if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
#         return solver.ObjectiveValue()
#     else:
#         return float('inf')

# # Define the objective function for Hyperopt. It receives a dictionary of parameters.
# def objective(params):
#     # Convert the parameters dictionary into a tuple of weights.
#     weights = (
#         params['w_consec'],
#         params['w_start'],
#         params['w_end'],
#         params['w_carry'],
#         params['w_opp'],
#         params['w_high']
#     )
#     # Evaluate the model using these weights.
#     obj_val = evaluate_weights(num_teams, weights)
#     return obj_val

# # Define the search space for each weight using hp.choice.
# space = {
#     'w_consec': hp.choice('w_consec', [1, 3, 5, 7, 10]),
#     'w_start':  hp.choice('w_start',  [1, 3, 5]),
#     'w_end':    hp.choice('w_end',    [1, 3, 5]),
#     'w_carry':  hp.choice('w_carry',  [1, 3, 5]),
#     'w_opp':    hp.choice('w_opp',    [1, 2, 3]),
#     'w_high':   hp.choice('w_high',   [0, 1, 2])
# }

# # Set the number of teams for the instance. (You can adjust this as needed.)
# num_teams = 8

# # Create a Trials object to record the search history.
# trials = Trials()

# # Run the hyperparameter optimization using fmin.
# best = fmin(
#     fn=objective,
#     space=space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=trials
# )

# print("Best weight combination:", best)


In [18]:
schedule = belgian_pro_league_schedule(num_teams)


Starting CP-SAT solver v9.12.4544
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 8

Initial optimization model '': (model_fingerprint: 0x22ab278f050ce16f)
#Variables: 896 (#bools: 98 #ints: 112 in objective)
  - 784 Booleans in [0,1]
  - 14 in [0,7]
  - 91 in [0,10]
  - 7 in [0,100]
#kLinearN: 539 (#terms: 10'822)

Starting presolve at 0.00s
  2.78e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.19e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  3.30e-05s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=112 
  6.24e-04s  0.00e+00d  [DetectDuplicateColumns] 
  1.46e-04s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=21 
[Symmetry] Graph for symmetry has 2'016 nodes and 10'836 arcs.
[Symmetry] Symmetry computation done. time: 0.00438 dtime: 0.0105318
[Symmetry] #generators: 6, average support size: 518
[Symmetry] 49 orbits on 784 variables with sizes: 60,60,60,60,60,60,60,12,12,12,...
[Symmet

In [10]:
# # Display the schedule
# def display_schedule(schedule):
#     print("\n**Belgian Pro League - Match Schedule**\n")
#     for round_num, matches in enumerate(schedule, start=1):
#         print(f"**Round {round_num}**")
#         for match in matches:
#             if match[0] == "Bye":
#                 print(f"Team {match[1]} has a Bye")
#             else:
#                 print(f"Team {match[0]} (Home) vs. Team {match[1]} (Away)")
#         print("-" * 30)

# display_schedule(schedule)

In [21]:
import pandas as pd

def display_schedule_dataframe(schedule, teams):
    """
    Constructs and returns a DataFrame showing the schedule with team rankings.
    
    Each round is shown with matches:
      - For a match tuple (i, j): displays "Team i (ranking) (Home) vs. Team j (ranking) (Away)"
      - For a bye tuple ("Bye", team): displays "Team {team} (ranking) has a Bye"
    """
    rows = []
    for round_idx, matches in enumerate(schedule, start=1):
        for match in matches:
            if match[0] == "Bye":
                team = match[1]
                row = {
                    "Round": round_idx,
                    "Home Team": "",
                    "Home Ranking": "",
                    "Match": f"Team {team} ({teams[team]}) has a Bye",
                    "Away Team": "",
                    "Away Ranking": ""
                }
            else:
                home, away = match
                row = {
                    "Round": round_idx,
                    "Home Team": f"Team {home}",
                    "Home Ranking": teams[home],
                    "Match": f"Team {home} ({teams[home]}) (Home) vs. Team {away} ({teams[away]}) (Away)",
                    "Away Team": f"Team {away}",
                    "Away Ranking": teams[away]
                }
            rows.append(row)
    df = pd.DataFrame(rows)
    return df


df_schedule = display_schedule_dataframe(schedule, teams)


In [22]:
df_schedule

,Round,Home Team,Home Ranking,Match,Away Team,Away Ranking
0,1,Team 1,3,Team 1 (3) (Home) vs. Team 2 (6) (Away),Team 2,6
1,1,Team 4,5,Team 4 (5) (Home) vs. Team 3 (1) (Away),Team 3,1
2,1,Team 5,7,Team 5 (7) (Home) vs. Team 0 (2) (Away),Team 0,2
3,1,,,Team 6 (4) has a Bye,,
4,2,Team 2,6,Team 2 (6) (Home) vs. Team 5 (7) (Away),Team 5,7
5,2,Team 3,1,Team 3 (1) (Home) vs. Team 1 (3) (Away),Team 1,3
6,2,Team 6,4,Team 6 (4) (Home) vs. Team 4 (5) (Away),Team 4,5
7,2,,,Team 0 (2) has a Bye,,
8,3,Team 1,3,Team 1 (3) (Home) vs. Team 0 (2) (Away),Team 0,2
9,3,Team 5,7,Team 5 (7) (Home) vs. Team 4 (5) (Away),Team 4,5


In [23]:
def print_debug_info(solver, num_teams, num_rounds, teams, v_consec, v_start, v_end, v_carry, v_opp, v_high):
    """
    Prints team rankings and soft constraint violation values.
    
    Parameters:
      - solver: the CP-SAT solver instance after solving the model
      - num_teams: number of teams
      - num_rounds: number of rounds in the schedule
      - teams: dictionary mapping team IDs to their rankings
      - v_consec: dictionary of binary violation variables for consecutive match violations
      - v_start: dictionary of binary violation variables for start fairness
      - v_end: dictionary of binary violation variables for end fairness
      - v_carry: dictionary of integer violation variables for carry-over effect violations
      - v_opp: dictionary of integer violation variables for opponent quality distribution violations
      - v_high: dictionary of integer violation variables for high-profile match distribution violations
    """
    print("\n--- Debug Information ---\n")
    
    # Print team rankings
    print("Team Rankings:")
    for i in range(num_teams):
        print(f"Team {i}: Ranking {teams[i]}")
    
    print("\nSoft Constraint Violations:")
    
    # Excessive Consecutive Matches Violations
    print("\nExcessive Consecutive Matches Violations:")
    for i in range(num_teams):
        for r in range(num_rounds - 2):
            violation = solver.Value(v_consec[i, r])
            if violation != 0:
                print(f"Team {i}, Rounds {r}-{r+2}: v_consec = {violation}")
    
    # Start Fairness Violations
    print("\nStart Fairness Violations:")
    for i in range(num_teams):
        print(f"Team {i}: v_start = {solver.Value(v_start[i])}")
    
    # End Fairness Violations
    print("\nEnd Fairness Violations:")
    for i in range(num_teams):
        print(f"Team {i}: v_end = {solver.Value(v_end[i])}")
    
    # Carry-Over Effect Violations
    print("\nCarry-Over Effect Violations:")
    for i in range(num_teams):
        for r in range(num_rounds - 1):
            violation = solver.Value(v_carry[i, r])
            if violation != 0:
                print(f"Team {i}, Rounds {r} & {r+1}: v_carry = {violation}")
    
    # Opponent Quality Distribution Violations
    print("\nOpponent Quality Distribution Violations:")
    for i in range(num_teams):
        print(f"Team {i}: v_opp = {solver.Value(v_opp[i])}")
    
    # High-Profile Match Distribution Violations (Optional)
    print("\nHigh-Profile Match Distribution Violations:")
    for r in range(num_rounds):
        violation = solver.Value(v_high[r])
        if violation != 0:
            print(f"Round {r}: v_high = {violation}")

In [24]:
print_debug_info(solver, num_teams, num_rounds, teams, v_consec, v_start, v_end, v_carry, v_opp, v_high)



--- Debug Information ---

Team Rankings:
Team 0: Ranking 2
Team 1: Ranking 3
Team 2: Ranking 6
Team 3: Ranking 1
Team 4: Ranking 5
Team 5: Ranking 7
Team 6: Ranking 4

Soft Constraint Violations:

Excessive Consecutive Matches Violations:

Start Fairness Violations:
Team 0: v_start = 0
Team 1: v_start = 0
Team 2: v_start = 0
Team 3: v_start = 0
Team 4: v_start = 0
Team 5: v_start = 0
Team 6: v_start = 0

End Fairness Violations:
Team 0: v_end = 0
Team 1: v_end = 0
Team 2: v_end = 0
Team 3: v_end = 0
Team 4: v_end = 0
Team 5: v_end = 0
Team 6: v_end = 0

Carry-Over Effect Violations:

Opponent Quality Distribution Violations:
Team 0: v_opp = 0
Team 1: v_opp = 0
Team 2: v_opp = 0
Team 3: v_opp = 0
Team 4: v_opp = 0
Team 5: v_opp = 0
Team 6: v_opp = 0

High-Profile Match Distribution Violations:
